## Zadanie klasyfikacja

In [1]:
import numpy as np
import pandas as pd


In [2]:
class Network:
    def __init__(self, eta, layers, max_i=10000, max_error=0.00001):
        self._eta = eta
        self._layers = layers
        self._w = []
        self._y = []
        self._delta = []
        self.max_i = max_i
        self.max_error = max_error

    def sigmoid(self, x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    def _create_weights(self, x_num):
        for i, layer_i in enumerate(self._layers):
            if i == 0:
                self._w.append(
                    np.random.uniform(-1, 1, size=(layer_i, x_num + 1)))
            else:
                self._w.append(
                    np.random.uniform(
                        -1, 1, size=(layer_i, self._w[i - 1].shape[0] + 1)
                    )
                )
            self._y.append(np.zeros(layer_i))
            self._delta.append(np.zeros(layer_i))

    def _forward_progation(self, x):
        for i in range(len(self._layers)):
            if i == 0:
                self._y[i] = np.dot(self._w[i], np.append(np.array(1), x))
            else:
                self._y[i] = np.dot(self._w[i], np.append(
                    np.array(1), self._y[i - 1]))
            self._y[i] = self.sigmoid(self._y[i])

    def _backward_propagation(self, expected_output):
        for i in range(len(self._y) - 1, -1, -1):
            if i == len(self._y) - 1:
                error = expected_output - self._y[i]
                self._delta[i] = error * self.sigmoid(self._y[i], True)
            else:
                self._delta[i] = np.dot(
                    self._w[i+1][:, 1:].T, self._delta[i+1]) * self.sigmoid(self._y[i], True)
        return error

    def _update(self, x):
        for i in range(len(self._layers)):
            for j in range(len(self._delta[i])):
                if i == 0:
                    self._w[i][j] += np.append([1], x) * \
                        self._delta[i][j] * self._eta
                else:
                    self._w[i][j] += (
                        np.append([1], self._y[i - 1]) *
                        self._delta[i][j] * self._eta
                    )

    def fit(self, x, y, verbose=False):
        self._create_weights(np.shape(x)[1])
        data = list(zip(x, y))
        for epoch in range(self.max_i):
            np.random.shuffle(data)
            E = 0
            for x_i, y_i in data:
                self._forward_progation(x_i)
                b = self._backward_propagation(y_i)
                b = b**2
                E += 0.5 * b.sum()
                self._update(x_i)
            if E < self.max_error:
                break
            if verbose:
                print(f"Epoka: {epoch+1} z {self.max_i}, blad: {E}")
        return E

    def predict(self, x):
        self._forward_progation(x)
        return self._y[-1]


In [3]:
df = pd.read_csv("data/iris.data", header=None)


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder

split = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
for train_index, test_index in split.split(df, df[4]):
    x_train, y_train = df.loc[train_index,
                              :3].to_numpy(), df.loc[train_index, 4]
    x_test, y_test = df.loc[test_index, :3].to_numpy(), df.loc[test_index, 4]

cat_encoder = OneHotEncoder()
y_train = cat_encoder.fit_transform(pd.DataFrame(y_train)[[4]]).toarray()
y_test = cat_encoder.fit_transform(pd.DataFrame(y_test)[[4]]).toarray()


In [5]:
networks = []
for i in range(1, 6):
    network = Network(0.2, (i, 3), 200, 0.01)
    network.fit(x_train, y_train)
    networks.append(network)


In [6]:
result = []
for j in range(5):
    result_pred = []
    for i in range(len(x_test)):
        y_pred = networks[j].predict(x_test[i])
        y_result = np.zeros(3)
        y_result[np.argmax(y_pred)] = 1
        result_pred.append(y_result)
    result.append(result_pred)


In [7]:
from sklearn.metrics import accuracy_score, roc_auc_score

for i in range(5):
    print(networks[i]._layers)
    print(f"Accuracy: {accuracy_score(y_test, result[i])}")
    print(f"roc_auc_score: {roc_auc_score(y_test, result[i])}")


(1, 3)
Accuracy: 0.8666666666666667
roc_auc_score: 0.9
(2, 3)
Accuracy: 0.9833333333333333
roc_auc_score: 0.9874999999999999
(3, 3)
Accuracy: 0.9333333333333333
roc_auc_score: 0.9500000000000001
(4, 3)
Accuracy: 0.8666666666666667
roc_auc_score: 0.9
(5, 3)
Accuracy: 0.9666666666666667
roc_auc_score: 0.975
